# 데이터 전처리(merge with labeling data)

In [2]:
import pandas as pd

prepro_data_path = '../datasets/preprocessing/'

activity = pd.read_csv(prepro_data_path+'prepro_activity.csv', index_col=0)
sleep = pd.read_csv(prepro_data_path+'prepro_sleep.csv', index_col=0)
mmse = pd.read_csv(prepro_data_path+'sample_mmse.csv', index_col=0)

In [12]:
labeling_data_path = '../datasets/data/1.Training/labeling/'

activity_label = pd.read_csv(labeling_data_path+'1.걸음걸이/training_label.csv')
sleep_label = pd.read_csv(labeling_data_path+'2.수면/training_label.csv')
mmse_label = pd.read_csv(labeling_data_path+'3.인지기능/training_label.csv')

In [14]:
import data_merge

In [15]:
labeled_activity = data_merge.merge_data(activity, activity_label,0)
labeled_sleep = data_merge.merge_data(sleep, sleep_label,0)
labeled_mmse = data_merge.merge_data(mmse, mmse_label,1)

In [16]:
labeled_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EMAIL                              141 non-null    object 
 1   activity_cal_total                 141 non-null    float64
 2   activity_high                      141 non-null    float64
 3   activity_inactive                  141 non-null    float64
 4   activity_low                       141 non-null    float64
 5   activity_medium                    141 non-null    float64
 6   activity_met_min_inactive          141 non-null    float64
 7   activity_non_wear                  141 non-null    float64
 8   activity_rest                      141 non-null    float64
 9   activity_score                     141 non-null    float64
 10  activity_score_meet_daily_targets  141 non-null    float64
 11  activity_score_move_every_hour     141 non-null    float64

# XGBoost

In [43]:
import xgboost
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

## 1. activity

데이터 셋의 80%만 학습 데이터로 사용, 나머지는 테스트 셋으로 사용

In [80]:
train_num = int(len(labeled_activity)*0.8)
print(train_num)

112


전체 데이터를 x값, y값으로 나눔

In [81]:
x_df = labeled_activity.drop(['EMAIL', 'target'], axis=1)
print('x:', len(x_df))

#target 값 one-hot encoding
y_df = pd.get_dummies(labeled_activity['target'])
print('y:', len(y_df))

x: 141
y: 141


train, test 셋으로 나눈 후 nparray로 변환(`.values`)

In [82]:
train_X = x_df[:train_num].values
train_y = y_df[:train_num].values
test_X = x_df[train_num:].values
test_y = y_df[train_num:].values

target 값 count

In [83]:
print(labeled_activity['target'].value_counts())

target
CN     85
MCI    47
Dem     9
Name: count, dtype: int64


모델 생성

In [84]:
model = LogisticRegression()
model = OneVsRestClassifier(model)

모델 학습


In [85]:
model.fit(train_X, train_y)

print(model.estimators_)

[LogisticRegression(), LogisticRegression(), LogisticRegression()]


In [86]:
y_predict = model.predict(test_X)

In [88]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

f1 = round(f1_score(test_y, y_predict, average='micro'), ndigits= 5)
precision = round(precision_score(test_y, y_predict, average='macro'), ndigits=5)
recall = round(recall_score(test_y, y_predict, average='macro'), ndigits=5)
#auc = round(roc_auc_score(test_y, y_predict, multi_class='ovo'), ndigits= 5)
print(f'정밀도 : {precision}, 재현율:{recall},  fl:{f1}')

정밀도 : 0.2807, 재현율:0.20513,  fl:0.65306


/home/sunooj/.pyenv/versions/3.12.0/envs/env3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sunooj/.pyenv/versions/3.12.0/envs/env3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
